In [16]:
from jupyter_dash import JupyterDash
import dash_daq as daq
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

import dash
import dash_cytoscape as cyto
import dash_html_components as html

import networkx as nx
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import json

In [19]:
filename = './example.json'

with open(filename) as f:
    js_graph = json.load(f)

# transform dict
node = js_graph['nodes']
link = js_graph['links']
for n in node:
    n['label']= n['id']
new_node = [{'data': i} for i in node]
new_link = [{'data':i} for i in link]

# get node order
node_list = []
for n in link:
    a = n['source']
    b = n['target']
    if a not in node_list:
        node_list.append(a)
    elif b not in node_list:
        node_list.append(b)
    else:
        pass

# calculate pos - better solution: dfs algorithm
x = 0; y = 0; c1 = 1
pos = []
for n in node_list: 
    if ('mixer' in n) or ('spliter' in n): # mix or seperate
        x = x + c1*200 
        pos.append([n,x,y])
        x = x - c1*200
        c1 = c1 * (-1)
    elif ('discharge' in n) or ('landfill' in n) or ('drinking' in n): # end use
        x = x + 200
        pos.append([n,x,y])
        x = x - 200
    else:
        y = y + 150
        pos.append([n,x,y])

# put pos into dict
df = pd.DataFrame(pos,columns=['node','x','y'])
for i in range(len(df)):
    name = new_node[i]['data']['label']
    row = df[df['node']==name]
    x = row['x'].values[0]
    y = row['y'].values[0]
    new_node[i]['position'] = {'x': y,'y': x} # change to horizontal

both = new_node + new_link

In [22]:
app = JupyterDash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-two-nodes',
        layout={'name': 'preset'},
        style={'width': '90%', 'height': '700px'},
        elements=both,
        stylesheet=[
            {'selector': 'edge','style': {
                'curve-style': 'bezier',
                'arrow-scale': 2,
                'target-arrow-shape': 'triangle'}},
            {'selector': 'node','style': {
                'label': 'data(id)',
                'background-color': '#1E90FF'}},
        ],
    )
])

if __name__ == '__main__':
    app.run_server(debug=True,port=8051)

Dash app running on http://127.0.0.1:8051/


In [19]:
df = px.data.tips()
fig = px.bar(df, x="sex", y="total_bill", color="smoker", barmode="group",
             facet_row="time", facet_col="day",
             category_orders={"day": ["Thur", "Fri", "Sat", "Sun"],
                              "time": ["Lunch", "Dinner"]})
fig.show()

In [49]:
df2 = pd.read_csv('./CarlsbadCompare.csv',index_col=0)
df3 = df2[df2['Metric'].isin(['Cost', 'Annual Cost'])]

In [109]:
df3['id'] = df3['Case Study']+' - '+ df3['Scenario']
pivot = pd.pivot_table(df3, values='Value', index=['Unit Process Name','Variable','Metric'], columns='id').reset_index()

df1 = pivot[['Unit Process Name','Variable','Metric','Carlsbad - Baseline']]
df2 = pivot[['Unit Process Name','Variable','Metric','Carlsbad - TwoPassRO']]
df1.rename(columns={'Carlsbad - Baseline':'Value'},inplace=True)
df2.rename(columns={'Carlsbad - TwoPassRO':'Value'},inplace=True)

df1_c = df1[df1['Metric']=='Cost']
df1_ac = df1[df1['Metric']=='Annual Cost']
df2_c = df1[df2['Metric']=='Cost']
df2_ac = df1[df2['Metric']=='Annual Cost']

fig = make_subplots(specs=[[{"secondary_y": True}]])

for var, group in df1_c.groupby('Variable'):
    fig.add_trace(go.Bar(x=group['Unit Process Name'], y=group['Value'], width=0.1, name='s1 -%s - %s'%('metric 1',var), offset=-0.2))

for var, group in df2_c.groupby('Variable'):
    fig.add_trace(go.Bar(x=group['Unit Process Name'], y=group['Value'], width=0.1, base=0, name='s2 -%s - %s'%('metric 2',var), offset=-0.1))
    
for var, group in df1_ac.groupby('Variable'):
    fig.add_trace(go.Bar(x=group['Unit Process Name'], y=group['Value'], width=0.1, name='s1 - %s - %s'%('metric 1',var), offset=0.05, marker_line_color='#595959', opacity=0.6), secondary_y=True)

for var, group in df2_ac.groupby('Variable'):
    fig.add_trace(go.Bar(x=group['Unit Process Name'], y=group['Value'], width=0.1, base=0, name='s2 -%s - %s'%('metric 2',var), offset=0.15, marker_line_color='#595959', opacity=0.6), secondary_y=True)

fig.update_layout(barmode='stack', width=1600, height=400)
fig.show()

In [1]:
l1 = ['a','b','c','d']
l2 = [6,7,8,9]

d3 = dict(zip(l1,l2))

In [2]:
d3

{'a': 6, 'b': 7, 'c': 8, 'd': 9}

In [3]:
d4 = {'a': 1, 'b': 2, 'c': 3}

In [10]:
[d3[i]+d4[i] for i in ['a','c']]

[7, 11]

In [8]:
list(d4.keys())

['a', 'b', 'c']

In [11]:
[d3[i] + d4[i] for i in list(d4.keys())]

[7, 9, 11]

In [146]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])

# for var, group in df1_c.groupby('Variable'):
#     fig.add_trace(go.Bar(x=group['Unit Process Name'], y=group['Value'], width=0.2, name='s1 -%s - %s'%('metric 1',var), offset=-0.1))

# for var, group in df1_ac.groupby('Variable'):
#     fig.add_trace(go.Scatter(x=group['Unit Process Name'], y=group['Value'], mode='markers', marker_color='#1E90FF', stackgroup=True, name='s1 - %s - %s'%('metric 1',var)), secondary_y=True)

# for var, group in df2_c.groupby('Variable'):
#     fig.add_trace(go.Bar(x=group['Unit Process Name'], y=group['Value'], width=0.2, base=0, name='s2 -%s - %s'%('metric 2',var), offset=0.1))

# for var, group in df2_ac.groupby('Variable'):
#     fig.add_trace(go.Scatter(x=group['Unit Process Name'], y=group['Value'], mode='markers', marker_color='#B22222', stackgroup=True, name='s2 -%s - %s'%('metric 2',var)), secondary_y=True)

# fig.update_layout(barmode='stack', width=1600, height=400)
# fig.show()

In [189]:
test = pd.DataFrame(columns={'Unit_Process'})

In [191]:
test['Unit_Process']

Series([], Name: Unit_Process, dtype: object)